# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Introduction

## Background

  Adverse drug-drug interaction (DDI) is the unintended molecular interactions between drugs. It’s a prominent cause of patient morbidities/mortalities, incurring higher costs and risking patient safety. The difficulty of mitigating this issue stems from a couple of factors: 

  * The molecular structure of drugs are complex, consisting of many units and substructures
  * Drug development is a process that requires highly specialized knowledge
  * Trials to test drugs and post-market surveillance are long and expensive processes

  With respect to applying ML to this topic, there are also a couple of issues:
  
  * There is a relatively light amount of training data that exists, due to the slow reporting of DDI instances
  * Deep learning models have a large number of parameters, making interpretation of the model’s results difficult. Thus, extracting the reasoning for why a DDI is occurring can be hard for researchers to glean.
  * DDIs usually result from the reactions of only a few sub-structures of a drug’s entire molecule, but many drug-drug pairs have significant overlaps on larger but irrelevant substructures. This skews the results of DDI prediction.

  There is major interest in predicting whether two drugs will interact (especially during the design process) to reduce testing/development costs and improving patient safety.

### The Current State of the Art
 
  Deep learning models have been successfully used to predict DDIs, however such previous models often generate drug representations using the entire chemical representation, causing learned representations to be potentially biased toward the large, irrelevant substructures and ultimately nullify learned drug similarities and predictions.

## CASTER
  The ChemicAl SubstrucTurE Representation framework, or CASTER, was introduced as a DDI prediction model, improving on the weaknesses of prior works.
  * what did the paper propose
  * what is the innovations of the method
  * how well the proposed method work (in its own metrics)
  * what is the contribution to the reasearch regime (referring the Background above, how important the paper is to the problem).


# Scope of Reproducibility:

The authors of CASTER presented the following hypotheses:

1.  The CASTER model will provide more accurate DDI predictions when compared with other established models.

2.  The usage of unlabelled data to generate frequent sub-structure features improves performance in situations with limited labeled datasets.

3.  CASTER’s sub-structure dictionary can help human operators better comprehend the final result.


# Methodology

In [1]:
#Just a bunch of imports

import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch import nn 
import copy

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.model_selection import KFold
torch.manual_seed(2)    # reproducible torch:2 np:3
np.random.seed(3)

#Used for parsing the CASTER dataset
from subword_nmt.apply_bpe import BPE
import codecs



##  Data
The data that the model ingests consists of 3 datasets:
  * unsup_dataset.csv = A dataset of randomly combined pairs of SMILEs strings drawn from FooDB (a db of food constituent molecules) and all drugs, drawn from DrugBank
    * This is the unsupervised dataset used to help find frequent SMILEs substructures
  * BIOSNAP/sup*.csv = A dataset from Stanford's Biomedical Network Dataset indicating pairs of SMILEs strings and presence of DDI
    * These are the supervised datasets
  * subword_units_map.csv = The 1722 frequent patterns extracted from the above strings, produced by the sequential pattern mining (SPM) routine described in CASTER
  
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

In [2]:
#===== DATASET DEFINITIONS =====

dataFolder = './data'

vocab_path = dataFolder + '/codes.txt'
bpe_codes_fin = codecs.open(vocab_path)
bpe = BPE(bpe_codes_fin, merges=-1, separator='')

#Get frequent substructures
vocab_map = pd.read_csv(dataFolder + '/subword_units_map.csv')
idx2word = vocab_map['index'].values
words2idx = dict(zip(idx2word, range(0, len(idx2word))))

#===== Helper functions =====
#Map the smiles strings into multi-hot representations of substructures.
def smiles2index(s1, s2):
    t1 = bpe.process_line(s1).split() #split
    t2 = bpe.process_line(s2).split() #split
    i1 = [words2idx[i] for i in t1] # index
    i2 = [words2idx[i] for i in t2] # index
    return i1, i2

#Combine both multi-hot representations into a single multi-hot (the functional representation)
def index2multi_hot(i1, i2):
    v_d = np.zeros(len(idx2word),)
    v_d[i1] = 1
    v_d[i2] = 1
    return v_d

#Final product, takes two smiles strings and turns them into the functional representation multi-hot.
def smiles2vector(s1, s2):
    i1, i2 = smiles2index(s1, s2)
    v_d = index2multi_hot(i1, i2)
    return v_d

#===== Datasets =====
class sup_data(data.Dataset):

    def __init__(self, list_IDs, labels, df_ddi):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.df = df_ddi
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        index = self.list_IDs[index]
        # Load data from sample and get multi-hot
        s1 = self.df.iloc[index].Drug1_SMILES
        s2 = self.df.iloc[index].Drug2_SMILES
        v_d = smiles2vector(s1, s2)
        #Get label
        y = self.labels[index]
        #Return combined multi-hot and its label.
        return v_d, y
    
class unsup_data(data.Dataset):

    def __init__(self, list_IDs, df):
        'Initialization'
        self.list_IDs = list_IDs
        self.df = df

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        index = self.list_IDs[index]
        s1 = self.df.iloc[index].input1_SMILES
        s2 = self.df.iloc[index].input2_SMILES
        v_d = smiles2vector(s1, s2)
        return v_d

In [3]:
df_unsup = pd.read_csv('data/unsup_dataset.csv', names = ['idx', 'input1_SMILES', 'input2_SMILES', 'type']).drop(0)# pairs dataframe input1_smiles, input2_smiles
df_ddi = pd.read_csv('data/BIOSNAP/sup_train_val.csv')  # ddi dataframe drug1_smiles, drug2_smiles

#Print some basic info about the datasets 
print("="*30)
print("TOTAL UNSUPERVISED SET")
print("="*30)
print("# of samples:",len(df_unsup))
print("# of drug-food pairs:",sum(df_unsup.type == "df_pair"))
print("# of drug-drug pairs:",sum(df_unsup.type == "dd_pair"))
print("="*30)
print("BIOSNAP TRAIN/VAL SET")
print("="*30)
print("# of samples:",len(df_ddi))
print("# of DDIs:",sum(df_ddi.label == 1.0))

#Do kfold
kf = KFold(n_splits = 8, shuffle = True, random_state = 3)

#Get the 1st fold index
fold_index = next(kf.split(df_ddi), None)

ids_unsup = df_unsup.index.values
partition_sup = {'train': fold_index[0], 'val': fold_index[1]}
labels_sup = df_ddi.label.values

LOADER_PARAMS = {
    'batch_size': 128,
    'shuffle': True,
    'num_workers': 0
    }

unsup_set = unsup_data(ids_unsup, df_unsup)
unsup_generator = data.DataLoader(unsup_set, **LOADER_PARAMS)

training_set = sup_data(partition_sup['train'], labels_sup, df_ddi)
training_generator_sup = data.DataLoader(training_set, **LOADER_PARAMS)

validation_set = sup_data(partition_sup['val'], labels_sup, df_ddi)
validation_generator_sup = data.DataLoader(validation_set, **LOADER_PARAMS)
print(len(training_set.__getitem__(0)[0]))

TOTAL UNSUPERVISED SET
# of samples: 441853
# of drug-food pairs: 220926
# of drug-drug pairs: 220927
BIOSNAP TRAIN/VAL SET
# of samples: 66432
# of DDIs: 33243
1722


##   Model

![caster_structure.png](img/caster_block_diagram.png)

### Architecture
The model consists of a few notable parts:
* Sequential Pattern Mining (SPM)
    * The SMILEs strings are initially passed through a sequential pattern mining process to find frequently occurring substructures
    * SMILEs strings are encoded into multi-hot vectors of frequent substructures and combined into a single functional representation (X)
* Encoder
    * The functional representation is passed through the encoder to make a latent feature vector (z)
    * The individual freq. substructures, as one-hot vectors, are also passed through the encoder to make a latent dictionary of substructures (b_1-k)
        * Note: This is equivalent to passing an identity matrix through the encoder.
* Predictor
    * The coefficients are passed through a standard fully connected NN.
    

The model includes the model definition which usually is a class, model training, and other necessary parts.
* Model architecture: layer number/size/type, activation function, etc
* Training objectives: loss function, optimizer, weight of each loss term, etc
* Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
* The code of model should have classes of the model, functions of model training, model validation, etc.
* If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

In [ ]:
#This defines the configuration of the model/training.
CONFIG = {}

CONFIG['batch_size'] = 256

#The width of the drug-drug pair's multi-hot functional representation
CONFIG['input_dim'] = 1722
CONFIG['batch_first'] = True
CONFIG['num_class'] = 2

#Learning rate
CONFIG['LR'] = 1e-3
CONFIG['train_epoch'] = 3
CONFIG['pretrain_epoch'] = 1

CONFIG['recon_threshold'] = 0.0005 # change later

#===== Encoder/Decoder Parameters =====
CONFIG['encode_fc1_dim'] = 500  # encoder fc1
CONFIG['encode_fc2_dim'] = 50  # encoder fc2
CONFIG['decode_fc1_dim'] = 500  # decoder fc1
CONFIG['decode_fc2_dim'] = CONFIG['input_dim']  # decoder reconstruction

#===== Deep Predictor Parameters =====
CONFIG['predict_dim'] = 1024 # for every layer
CONFIG['predict_out_dim'] = 1 # predictor out

CONFIG['lambda1'] = 1e-2  # L1 regularization coefficient
CONFIG['lambda2'] = 1e-1  # L2 regulatization coefficient
CONFIG['lambda3'] = 1e-5  # L2 regulatization coefficient

CONFIG['reconstruction_coefficient'] = 1e-1  # 1e-2
CONFIG['projection_coefficient'] = 1e-1  # 1e-2
CONFIG['magnify_factor'] = 100

In [2]:
class CASTER(nn.Sequential):

    def __init__(self, **config):
        super(CASTER, self).__init__()
        self.input_dim = config['input_dim']
        self.num_class = config['num_class']
        self.lambda3 = config['lambda3']        
        self.encode_fc1_dim = config['encode_fc1_dim']
        self.encode_fc2_dim = config['encode_fc2_dim']
        self.decode_fc1_dim = config['decode_fc1_dim']
        self.decode_fc2_dim = config['decode_fc2_dim']
        self.predict_dim = config['predict_dim']
        self.predict_out_dim = config['predict_out_dim']
        self.mag_factor = config['magnify_factor']        

        # encoder: two layer NN
        self.encoder = nn.Sequential(
            nn.Linear(self.input_dim, self.encode_fc1_dim),
            nn.ReLU(True),
            nn.Linear(self.encode_fc1_dim, self.encode_fc2_dim)
        )
        # decoder: two layer NN
        self.decoder = nn.Sequential(
            nn.Linear(self.encode_fc2_dim, self.decode_fc1_dim),
            nn.ReLU(True),
            nn.Linear(self.decode_fc1_dim, self.decode_fc2_dim)
        )

        # predictor: eight layer NN
        self.predictor = nn.Sequential(
            # layer 1
            nn.Linear(self.input_dim, self.predict_dim),
            nn.ReLU(True),
            # layer 2
            nn.BatchNorm1d(self.predict_dim),
            nn.Linear(self.predict_dim, self.predict_dim),
            nn.ReLU(True),
            # layer 3
            nn.BatchNorm1d(self.predict_dim),
            nn.Linear(self.predict_dim, self.predict_dim),
            nn.ReLU(True),
            # layer 4
            nn.BatchNorm1d(self.predict_dim),
            nn.Linear(self.predict_dim, self.predict_dim),
            nn.ReLU(True),
            # layer 5
            nn.BatchNorm1d(self.predict_dim),
            nn.Linear(self.predict_dim, self.predict_dim),
            nn.ReLU(True),
            # layer 6
            nn.BatchNorm1d(self.predict_dim),
            nn.Linear(self.predict_dim, 64),
            nn.ReLU(True),
            # output layer
            nn.Linear(64, self.predict_out_dim)
        )

def dictionary_encoder(self, Z_D, Z_f):
    '''
    :param Z_D: batch_size x encode_fc2_dim
    :param Z_f: encode_fc2_dim x eta
    :return: sparse code X_o: batch_size x eta
    '''       
    
    DTD = torch.matmul(Z_f, Z_f.transpose(2, 1))  # D is Dictionary;  D^T D encode_dim x eta
    DTD_inv = torch.inverse(DTD + self.lambda3 * torch.eye(self.input_dim))  # (D^T D + \lambda2 I )^{-1} D^T D, eta x eta
    DTD_inv_DT = torch.matmul(DTD_inv, Z_f)  

    r = Z_D[:,None,:].matmul(DTD_inv_DT.transpose(2, 1)).squeeze(1) # batch_size x eta    
    return r

def forward(self, x_of_pair):
    '''
    :param x_of_pair: batch_size x width of multi-hot functional
    :return: recon, score, code
    '''
    _, x_width = x_of_pair.shape
    
    # Encode functional representation into latent representation 
    z_of_pair = self.encoder(x_of_pair)
    
    # Create latent dictionary
    b = self.encoder(torch.eye(x_width))
    
    b = b.mul(x_of_pair[:,:,None]) 

    # Decode latent representation
    recon_temp = self.decoder(z_of_pair)
    reconstructed = torch.sigmoid(recon_temp)
    
    # dictionary learning
    code = self.dictionary_encoder(z_of_pair, b)

    # Pass coeffs through the deep predictor
    score = self.predictor(self.mag_factor * code)
    return reconstructed, code, score, b, z_of_pair

#Instantiate Model
model = CASTER(CONFIG)
loss_func = None
log_reg_optimizer = None

def train_model_one_iter(model, loss_func, optimizer):
    pass

num_epoch = 10
# model training loop: it is better to print the training/validation losses during the training
for i in range(num_epoch):
    train_model_one_iter(model, loss_func, log_reg_optimizer)
    train_loss, valid_loss = None, None
    print("Train Loss: %.2f, Validation Loss: %.2f" % (train_loss, valid_loss))

NameError: name 'nn' is not defined

## Ablation Model: Logistic Regression

One of the ablations we proposed (and one that was also suggested in the paper) was using logistic regression (LR) instead of CASTER's deep predictor. Specifically, the pair's functional representation (sub-structured/post pattern-mined) is classified using LR. According to the paper, though far lighter in parameter count, it was not chosen due its weaker performance.

In [8]:
class simple_log_reg(nn.Module):

    def __init__(self, n_inputs, n_outputs):
        super(simple_log_reg, self).__init__()
        self.linear = torch.nn.Linear(n_inputs, n_outputs)
        self.double()
    
    def forward(self,x):
        pred = torch.sigmoid(self.linear(x))
        return pred[:,-1]

n_inputs = CONFIG["input_dim"] #Width of the multi-hot vector
n_outputs = 1
log_reg = simple_log_reg(n_inputs, n_outputs)

# defining the optimizer
log_reg_optimizer = torch.optim.SGD(log_reg.parameters(), lr=0.001)
# defining Cross-Entropy loss
log_reg_criterion = torch.nn.BCELoss()

epochs = 50
Loss = []
acc = []
print("Starting training")
for epoch in range(epochs):
    # print("epoch:",epoch)
    for x_of_pair, y in training_generator_sup:
        # print(y)
        log_reg_optimizer.zero_grad()
        # print(x)
        # print(x.shape)
        y_pred = log_reg(x_of_pair)
        loss = log_reg_criterion(y_pred, y)
        # Loss.append(loss.item())
        loss.backward()
        log_reg_optimizer.step()
    Loss.append(loss.item())
    correct = 0
    # for x, y in validation_generator_sup:
    #     y_pred = log_reg(x)
    #     # print(y_pred)
    #     predicted = y_pred > 0.5
    #     correct = (predicted == y).sum()
    # accuracy = 100 * (correct.item()) / len(y)
    # acc.append(accuracy)
    print('Epoch: {}. Loss: {}. Accuracy: {}'.format(epoch, loss.item(), 0))
print("Done training LR.")

Starting training
Epoch: 0. Loss: 0.6749600020899327. Accuracy: 0
Epoch: 1. Loss: 0.6640584605572155. Accuracy: 0
Epoch: 2. Loss: 0.648485678074174. Accuracy: 0
Epoch: 3. Loss: 0.7013468194961268. Accuracy: 0
Epoch: 4. Loss: 0.6657262232705916. Accuracy: 0
Epoch: 5. Loss: 0.6544603837170421. Accuracy: 0
Epoch: 6. Loss: 0.6249702524725425. Accuracy: 0
Epoch: 7. Loss: 0.6263930347633465. Accuracy: 0
Epoch: 8. Loss: 0.6580098734968453. Accuracy: 0
Epoch: 9. Loss: 0.6833581732996595. Accuracy: 0
Epoch: 10. Loss: 0.6631222575097829. Accuracy: 0
Epoch: 11. Loss: 0.6573572399505658. Accuracy: 0
Epoch: 12. Loss: 0.6518567596097673. Accuracy: 0
Epoch: 13. Loss: 0.6227891636092892. Accuracy: 0
Epoch: 14. Loss: 0.6992572614928463. Accuracy: 0
Epoch: 15. Loss: 0.6175400640164673. Accuracy: 0
Epoch: 16. Loss: 0.6332888308638058. Accuracy: 0
Epoch: 17. Loss: 0.632756286974647. Accuracy: 0
Epoch: 18. Loss: 0.5793076304849437. Accuracy: 0
Epoch: 19. Loss: 0.6134011414760278. Accuracy: 0
Epoch: 20. Los

# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)

To begin the evaluation process, we iterate through a test data set using a data loader to obtain the predictions (y_pred) from the model for each input (v_D). We then use the ground truth labels (y_label) to which we compare our predictions to calculate the evaluation metrics.

The evaluation metrics used here include some of the most commonly used metrics to assess the performance of binary models: 

* **Average Precision Score**: Measures the precision-recall trade-off by summarizing the precision-recall curve as a weighted mean of precisions achieved at various thresholds.

* **Area under the ROC Curve (AUC)**: This metric effectively describes the trade-off between true-positive rate and false-positive rate. It essentially quantifies the ability of the model to distingush between classes.




In [ ]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

from matplotlib import pyplot as plt
params = {'batch_size': 256,
              'shuffle': True,
              'num_workers': 6}

dataFolder = './data'

df_ddi = pd.read_csv('../data/SNAP/sup_test.csv')  # ddi dataframe drug1_smiles, drug2_smiles
labels_sup = df_ddi.label.values
test_set = supData(df_ddi.index.values, labels_sup, df_ddi)
test_generator_sup = data.DataLoader(test_set, **params)

model_nn = model_max

y_pred = []
y_label = []
model_nn.eval()
for i, (v_D, label) in tqdm(enumerate(test_generator_sup)):
    recon, code, score, Z_f, z_D = model_nn(v_D.float())
    m = torch.nn.Sigmoid()
    logits = torch.squeeze(m(score)).detach().cpu().numpy()
    label_ids = label.to('cpu').numpy()
    y_label = y_label + label_ids.flatten().tolist()
    y_pred = y_pred + logits.flatten().tolist()

In the following cells, we will plot the precision-recall curve and the AUC described above. The precision-recall curve demonstrates the precision of the model at various recall levels. This gives us an idea of the model's ability to identify positive occurences with false positives minimized.

Overall, using these metrics and evaluations, we get a clear picture of the model's performance across different thresholds.

In [ ]:
from sklearn.metrics import average_precision_score
average_precision_score(y_label, y_pred)
from sklearn.metrics import precision_recall_curve
from sklearn.utils.fixes import signature
average_precision = average_precision_score(y_label, y_pred)
precision, recall, _ = precision_recall_curve(y_label, y_pred)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
average_precision

In [ ]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc, confusion_matrix, classification_report

fpr, tpr, thresholds = roc_curve(y_label, y_pred)
auc_score = auc(fpr, tpr)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Val (area = {:.3f})'.format(auc_score))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')

# Model Comparison

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can read and plot it here like the Scope of Reproducibility
'''

# References

1.   Huang, K., Xiao, C., Hoang, T., Glass, L., & Sun, J. (2020, April). Caster: Predicting drug interactions with chemical substructure representation. In Proceedings of the AAAI conference on artificial intelligence (Vol. 34, No. 01, pp. 702-709).

